In [1]:
from elasticsearch import Elasticsearch
from itertools import combinations
from os import listdir
import numpy as np
import pandas as pd
import TFIDFViewer as tfidf
from ssl import create_default_context
# context = create_default_context(cafile="/home/mohana/http_ca.crt")
client = Elasticsearch(timeout=1000,
                       # use_ssl=True,
                       # ssl_context=context,
                       # scheme = "https",
                       # http_auth = ("elastic", "h_9RfOqHmS-v5-ZV9Lk7")
                       )


# TF-IDF and cosine similarity

In [2]:
# create indexes
%run IndexFilesPreprocess.py --path ./arxiv_abs --index arxiv_letter_lowercase --token letter --filter lowercase
%run IndexFilesPreprocess.py --path ./20_newsgroups --index news_letter_lowercase --token letter --filter lowercase
%run IndexFilesPreprocess.py --path ./novels --index novel_letter_lowercase --token letter --filter lowercase
%run IndexFilesPreprocess.py --path ./docs --index docs_letter_lowercase --token letter --filter lowercase

Indexing 55760 files
Reading files ...


/home/tanja/anaconda3/envs/irrs/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: the default value for the ?wait_for_active_shards parameter will change from '0' to 'index-setting' in version 8; specify '?wait_for_active_shards=index-setting' to adopt the future default behaviour, or '?wait_for_active_shards=0' to preserve today's behaviour
  warnings.warn(message, category=ElasticsearchWarning)
/home/tanja/anaconda3/envs/irrs/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Using include_type_name in put mapping requests is deprecated. The parameter will be removed in the next major version.
  warnings.warn(message, category=ElasticsearchWarning)


Index settings= {'arxiv_letter_lowercase': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'arxiv_letter_lowercase', 'creation_date': '1665512278550', 'analysis': {'analyzer': {'default': {'filter': ['lowercase'], 'type': 'custom', 'tokenizer': 'letter'}}}, 'number_of_replicas': '1', 'uuid': 'XRspssThSvWNYGP9MAzjpQ', 'version': {'created': '7170699'}}}}}
Indexing ...
Indexing 20089 files
Reading files ...
Index settings= {'news_letter_lowercase': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'news_letter_lowercase', 'creation_date': '1665512322062', 'analysis': {'analyzer': {'default': {'filter': ['lowercase'], 'type': 'custom', 'tokenizer': 'letter'}}}, 'number_of_replicas': '1', 'uuid': '4zF-lsUURGCYvsSQoXbs1w', 'version': {'created': '7170699'}}}}}
Indexing ...
Indexing 33 files
Reading files

Compare similarity of a file with itself. Which is expected to be 1, because both files are literally equal.

In [3]:
%run TFIDFViewer.py --index novel_letter_lowercase --files ./novels/DickensThePickwickPapers.txt  ./novels/DickensThePickwickPapers.txt

Similarity = 1.00000


Use docs data set from lecture, to test implementation

In [4]:
%run TFIDFViewer.py --index docs_letter_lowercase --files ./docs/3 ./docs/4

Similarity = 0.03505


### Experimenting

Compare similarity from similar domain (arxiv data set) vs. from different domains

In [5]:
%run TFIDFViewer.py --index arxiv_letter_lowercase --files ./arxiv_abs/math.updates.on.arXiv.org/000000 ./arxiv_abs/math.updates.on.arXiv.org/000001

Similarity = 0.01441


In [6]:
%run TFIDFViewer.py --index arxiv_letter_lowercase --files ./arxiv_abs/cs.updates.on.arXiv.org/000000 ./arxiv_abs/cs.updates.on.arXiv.org/000001

Similarity = 0.02587


In [7]:
%run TFIDFViewer.py --index arxiv_letter_lowercase --files ./arxiv_abs/physics.updates.on.arXiv.org/000000 ./arxiv_abs/physics.updates.on.arXiv.org/000001

Similarity = 0.05931


In [8]:
# different domain
%run TFIDFViewer.py --index arxiv_letter_lowercase --files ./arxiv_abs/physics.updates.on.arXiv.org/000000 ./arxiv_abs/cs.updates.on.arXiv.org/000001

Similarity = 0.02292


In [9]:
# different domain
%run TFIDFViewer.py --index arxiv_letter_lowercase --files ./arxiv_abs/math.updates.on.arXiv.org/000000 ./arxiv_abs/cs.updates.on.arXiv.org/000001

Similarity = 0.06139


compare novels from same or different author

In [10]:
# same author
%run TFIDFViewer.py --index novel_letter_lowercase --files ./novels/DickensAChristmasCarol.txt ./novels/DickensGreatExpectations.txt

Similarity = 0.01716


In [11]:
# same author
%run TFIDFViewer.py --index novel_letter_lowercase --files ./novels/PoeWorksVol1.txt ./novels/PoeWorksVol2.txt

Similarity = 0.24176


In [12]:
# different author
%run TFIDFViewer.py --index novel_letter_lowercase --files ./novels/DickensAChristmasCarol.txt ./novels/LondonCallofTheWild.txt

Similarity = 0.00456


In [13]:
# different author
%run TFIDFViewer.py --index novel_letter_lowercase --files ./novels/DickensAChristmasCarol.txt ./novels/DarwinOriginofSpecies.txt

Similarity = 0.00277


newsgroup data set: compare subsets

In [14]:
# different subset
%run TFIDFViewer.py --index news_letter_lowercase --files ./20_newsgroups/alt.atheism/0000000 ./20_newsgroups/comp.os.ms-windows.misc/0002000

Similarity = 0.00676


In [15]:
# same subset
%run TFIDFViewer.py --index news_letter_lowercase --files ./20_newsgroups/comp.os.ms-windows.misc/0002000 ./20_newsgroups/comp.os.ms-windows.misc/0002001

Similarity = 0.00152


In [16]:
# same subset
%run TFIDFViewer.py --index news_letter_lowercase --files ./20_newsgroups/comp.os.ms-windows.misc/0002000 ./20_newsgroups/comp.os.ms-windows.misc/0002010

Similarity = 0.00626


In [17]:
# same subset
%run TFIDFViewer.py --index news_letter_lowercase --files ./20_newsgroups/talk.politics.guns/0011926 ./20_newsgroups/talk.politics.guns/0011939

Similarity = 0.03807


In [18]:
# similar subset: politics
%run TFIDFViewer.py --index news_letter_lowercase --files ./20_newsgroups/talk.politics.guns/0011926 ./20_newsgroups/talk.politics.mideast/0016011

Similarity = 0.02426


In [19]:
# similar subset: autos and motorcycles
%run TFIDFViewer.py --index news_letter_lowercase --files ./20_newsgroups/rec.autos/0006050 ./20_newsgroups/rec.motorcycles/0008000

Similarity = 0.00086


## Compare texts within groups

After experimenting with only single combination of files, we should use more files. We use the newsgroup data set for this analysis. In newspapers we have 20 groups:['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc'] and in each of these groups we will select 5 random texts to compare comparision in each group. At the end we have 200 combination of texts in different news subjects.

In [20]:
def within_groups_samples(path, n_sample):
    groups = sorted(listdir(path))
    group_text1_text2 = [[] for _ in range(3)]
    for i in groups:
        samples = np.random.choice(listdir(path +'/'+ i), size=n_sample, replace=False)
        #consider all combination of texts, forexample([0085,0147],[0085, 0524])
        for text1, text2 in combinations(samples, 2):
            group_text1_text2[0].append(i)
            group_text1_text2[1].append(text1)
            group_text1_text2[2].append(text2)
    df_within_groups = pd.DataFrame({'Group': group_text1_text2[0], 'Text1': group_text1_text2[1], 'Text2': group_text1_text2[2]})
    return df_within_groups


In [21]:
%run IndexFilesPreprocess.py --path ./20_newsgroups --index news --token letter --filter lowercase

Indexing 20089 files
Reading files ...
Index settings= {'news': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'news', 'creation_date': '1665512357048', 'analysis': {'analyzer': {'default': {'filter': ['lowercase'], 'type': 'custom', 'tokenizer': 'letter'}}}, 'number_of_replicas': '1', 'uuid': 'eYk6IrkDQgm5QWXTtsxnHQ', 'version': {'created': '7170699'}}}}}
Indexing ...


In [22]:
path_news = '20_newsgroups'
n_sample = 5
df_total_within = within_groups_samples(path_news, n_sample)
df_total_within

,Group,Text1,Text2
0,alt.atheism,0000147,0000372
1,alt.atheism,0000147,0000838
2,alt.atheism,0000147,0000433
3,alt.atheism,0000147,0000259
4,alt.atheism,0000372,0000838
...,...,...,...
195,talk.religion.misc,0019810,0019194
196,talk.religion.misc,0019810,0019167
197,talk.religion.misc,0000659,0019194
198,talk.religion.misc,0000659,0019167


## Compare texts between groups

To compare different groups, we will consider all comination of groups and for each group, we will select 10 random texts for each of them. To not have repetitive samples in our comparison we considered different random.seed for each iteration of combinations.At the end we have 950 combinations. 

In [23]:
def between_groups_samples(path, n_sample):
    groups = sorted(listdir(path))
    group1_text1_group2_text2 = [[] for _ in range(4)]
    counter = 0
    #All combinations of groups
    for group1, group2 in combinations(groups, 2):
        np.random.seed(1 + counter)
        sample_group1 = np.random.choice(listdir(path +'/' + group1), size=n_sample, replace=False)
        sample_group2 = np.random.choice(listdir(path +'/' + group2), size=n_sample, replace=False)
        for i in range(n_sample):
            group1_text1_group2_text2[0].append(group1)
            group1_text1_group2_text2[1].append(sample_group1[i])
            group1_text1_group2_text2[2].append(group2)
            group1_text1_group2_text2[3].append(sample_group2[i])
        counter += 1

    df_between_groups = pd.DataFrame({'Group1': group1_text1_group2_text2[0], 'Text1': group1_text1_group2_text2[1], 
                                      'Group2': group1_text1_group2_text2[2], 'Text2': group1_text1_group2_text2[3]})
    return df_between_groups

In [24]:
df_total_btw = between_groups_samples(path_news, n_sample)
print(df_total_btw)
#to check different combinations in group alt.atheism
#df_total_btw.loc[df_total_btw['Group1']=="alt.atheism"]

                 Group1    Text1              Group2    Text2
0           alt.atheism  0000432       comp.graphics  0001512
1           alt.atheism  0000821       comp.graphics  0001148
2           alt.atheism  0000911       comp.graphics  0001414
3           alt.atheism  0000664       comp.graphics  0001710
4           alt.atheism  0000240       comp.graphics  0001653
..                  ...      ...                 ...      ...
945  talk.politics.misc  0018105  talk.religion.misc  0019277
946  talk.politics.misc  0018096  talk.religion.misc  0019476
947  talk.politics.misc  0018897  talk.religion.misc  0019226
948  talk.politics.misc  0018785  talk.religion.misc  0019418
949  talk.politics.misc  0018179  talk.religion.misc  0000573

[950 rows x 4 columns]


## Calculate cosine similarity between groups

Based on functions that defined in TFIDFViewer, first we should find the id of a document in the index and then calculate cosine similarity.

In [25]:
def calculate_cos_sim(client_, index_, path1_, path2_):
    file_group1_id = tfidf.search_file_by_path(client_, index_, path1_)
    file_group2_id = tfidf.search_file_by_path(client_, index_, path2_)

    file1_tw = tfidf.toTFIDF(client_, index_, file_group1_id)
    file2_tw = tfidf.toTFIDF(client_, index_, file_group2_id )

    return tfidf.cosine_similarity(file1_tw, file2_tw)

We condider index news that considered token =letter and filter= lowercase

In [26]:
index = 'news'

For different combination of groups we will calculate mean of similarity, so we will have 190 rows at the end, because we consider mean of each group and our first table that has 950 rows.

In [27]:
%%capture
total_similarity = []
for ix, row in df_total_btw.iterrows():
    path1 = './' + path_news + '/' + row.Group1 + '/' + row.Text1
    path2 = './' + path_news + '/' + row.Group2 + '/' + row.Text2
    total_similarity.append(calculate_cos_sim(client, index, path1, path2))

between_groups_sim = df_total_btw.copy()
between_groups_sim['similarity'] = total_similarity
between_groups_sim = between_groups_sim.groupby(['Group1', 'Group2'])['similarity'].mean()

Based on our result, highest similarity exist between "talk.politics.guns" & "talk.politics.misc" and lowest similairty exist between "comp.windows.x" & "rec.autos".

In [28]:
df_between_sorted=pd.DataFrame(between_groups_sim).sort_values('similarity', ascending=False)
df_between_sorted

similarity
Group1                   Group2                            
comp.sys.ibm.pc.hardware sci.space                 0.038606
                         comp.sys.mac.hardware     0.038266
alt.atheism              talk.politics.misc        0.033682
sci.med                  soc.religion.christian    0.030710
soc.religion.christian   talk.politics.misc        0.028846
...                                                     ...
comp.sys.ibm.pc.hardware rec.autos                 0.004211
comp.graphics            talk.politics.mideast     0.004154
misc.forsale             talk.religion.misc        0.003392
                         soc.religion.christian    0.003003
                         rec.sport.baseball        0.002536

[190 rows x 1 columns]

## Calculate cosine similarity within groups

In [29]:
%%capture
nw_similarity = []
for ix, row in df_total_within.iterrows():
    path1 = './' + path_news + '/' + row.Group + '/' + row.Text1
    path2 = './' + path_news + '/' +row.Group + '/' + row.Text2
    nw_similarity.append(calculate_cos_sim(client, index, path1, path2))

within_groups_sim= df_total_within.copy()
within_groups_sim['similarity'] = nw_similarity
mean_within = within_groups_sim.groupby('Group')['similarity'].mean()
print(mean_within)

In [30]:
df_within_sorted = pd.DataFrame(mean_within).sort_values('similarity', ascending=False)
df_within_sorted

,similarity
Group,
talk.politics.mideast,0.092695
rec.autos,0.051515
talk.politics.guns,0.045533
soc.religion.christian,0.044261
rec.motorcycles,0.041701
talk.religion.misc,0.039863
rec.sport.hockey,0.032200
alt.atheism,0.031186
comp.windows.x,0.030744
